In [1]:
import os
import numpy, scipy, pandas
import matplotlib
import matplotlib.pyplot as plt
from imp import reload
import seaborn
import cython

import peas

d = numpy.arange(25).reshape((5,5)).astype(float)
print(d)

[[ 0.  1.  2.  3.  4.]
 [ 5.  6.  7.  8.  9.]
 [10. 11. 12. 13. 14.]
 [15. 16. 17. 18. 19.]
 [20. 21. 22. 23. 24.]]


In [2]:
%%time
%time st2 = peas.scoring_funcs_cython.compute_sum_table_2d_shuffled(d, 2, d.shape[0], 1)

CPU times: user 124 µs, sys: 0 ns, total: 124 µs
Wall time: 160 µs
CPU times: user 1.55 ms, sys: 0 ns, total: 1.55 ms
Wall time: 1.57 ms


In [3]:
# bed_fname = '/home/dskola/projects/coupled_peaks/moremerged/peak_matrices/atac_notx_peak_matrix.tsv'
# bed_df = pandas.read_csv(bed_fname, index_col=0, sep='\t')

# %time corrs = bed_df.iloc[:, 6:].T.corr()

In [4]:
%%time
hic_corrs_fname = '/home/dskola/projects/coupled_peaks/hic/c57_hic_corrs_chr19.tsv'
hic_corrs = pandas.read_csv(hic_corrs_fname, sep='\t', index_col=0)

CPU times: user 12.2 s, sys: 543 ms, total: 12.8 s
Wall time: 12.8 s


In [5]:
%time hic_corrs, _, _, _, _ = peas.interface.trim_data_matrix(hic_corrs.values)
hic_corrs = numpy.array(hic_corrs, order='c')

CPU times: user 136 ms, sys: 989 µs, total: 137 ms
Wall time: 134 ms


2018-Sep-05 14:24:06		trimmed 6134 x 6134 matrix to remove contiguous NaNs, now 5825 x 5825.


In [6]:
hic_corrs

array([[ 1.        ,  0.67820989,  0.30999799, ..., -0.00849356,
        -0.01138646, -0.01854142],
       [ 0.67820989,  1.        ,  0.71799514, ..., -0.01637958,
        -0.02055494, -0.03230393],
       [ 0.30999799,  0.71799514,  1.        , ..., -0.03087628,
        -0.03961263, -0.05143196],
       ...,
       [-0.00849356, -0.01637958, -0.03087628, ...,  1.        ,
         0.5600207 ,  0.04241341],
       [-0.01138646, -0.02055494, -0.03961263, ...,  0.5600207 ,
         1.        ,  0.61430029],
       [-0.01854142, -0.03230393, -0.05143196, ...,  0.04241341,
         0.61430029,  1.        ]])

## Benchmark

In [7]:
reload(peas.scoring)

<module 'peas.scoring' from '/gpfs/data01/glasslab/home/dskola/workspace/PEAS/peas/scoring.py'>

In [8]:
peas.scoring.constants.USE_C = False
%timeit peas.scoring.compute_sum_table_2d(hic_corrs, 2)

1.52 s ± 20.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
peas.scoring.constants.USE_C = True
%timeit peas.scoring.compute_sum_table_2d(hic_corrs, 2)

264 ms ± 6.24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
peas.scoring.constants.USE_C = False
%timeit peas.scoring.compute_mean_table_2d(hic_corrs, 2)

1.75 s ± 51.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
peas.scoring.constants.USE_C = True
%timeit peas.scoring.compute_mean_table_2d(hic_corrs, 2)

495 ms ± 29.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
reload(peas.scoring)
peas.scoring.constants.USE_C = True
%timeit peas.scoring.compute_mean_table_2d(hic_corrs, 2)

513 ms ± 19.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Combine with shuffling

In [13]:
%%timeit
numpy.random.seed(1)
%time shuffled_matrix = peas.arrayfuncs.shuffle_matrix(hic_corrs)
%time st1 = peas.scoring.compute_sum_table_2d(shuffled_matrix, 2)

CPU times: user 1.98 s, sys: 172 ms, total: 2.15 s
Wall time: 507 ms
CPU times: user 293 ms, sys: 103 ms, total: 396 ms
Wall time: 394 ms
CPU times: user 522 ms, sys: 106 ms, total: 628 ms
Wall time: 626 ms
CPU times: user 321 ms, sys: 107 ms, total: 428 ms
Wall time: 427 ms
CPU times: user 423 ms, sys: 109 ms, total: 532 ms
Wall time: 530 ms
CPU times: user 309 ms, sys: 99.9 ms, total: 409 ms
Wall time: 408 ms
CPU times: user 419 ms, sys: 103 ms, total: 522 ms
Wall time: 520 ms
CPU times: user 300 ms, sys: 107 ms, total: 407 ms
Wall time: 406 ms
CPU times: user 417 ms, sys: 103 ms, total: 520 ms
Wall time: 517 ms
CPU times: user 303 ms, sys: 104 ms, total: 407 ms
Wall time: 406 ms
CPU times: user 428 ms, sys: 103 ms, total: 531 ms
Wall time: 529 ms
CPU times: user 300 ms, sys: 110 ms, total: 410 ms
Wall time: 409 ms
CPU times: user 421 ms, sys: 101 ms, total: 522 ms
Wall time: 521 ms
CPU times: user 309 ms, sys: 101 ms, total: 410 ms
Wall time: 408 ms
CPU times: user 410 ms, sys: 111 

In [14]:
%%timeit
st2 = peas.scoring_funcs_cython.compute_sum_table_2d_shuffled(hic_corrs, 2, hic_corrs.shape[0], 1)

432 ms ± 3.21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Means shuffled

In [15]:
%%timeit
numpy.random.seed(1)
%time shuffled_matrix = peas.arrayfuncs.shuffle_matrix(hic_corrs)
%time mt1 = peas.scoring.compute_mean_table_2d(shuffled_matrix, 2)

CPU times: user 430 ms, sys: 109 ms, total: 539 ms
Wall time: 536 ms
CPU times: user 7.91 s, sys: 301 ms, total: 8.21 s
Wall time: 687 ms
CPU times: user 2.21 s, sys: 187 ms, total: 2.39 s
Wall time: 516 ms
CPU times: user 6.9 s, sys: 233 ms, total: 7.13 s
Wall time: 645 ms
CPU times: user 2.93 s, sys: 211 ms, total: 3.14 s
Wall time: 514 ms
CPU times: user 7.67 s, sys: 281 ms, total: 7.95 s
Wall time: 684 ms
CPU times: user 2.32 s, sys: 188 ms, total: 2.51 s
Wall time: 514 ms
CPU times: user 7.69 s, sys: 257 ms, total: 7.95 s
Wall time: 680 ms
CPU times: user 2.34 s, sys: 179 ms, total: 2.52 s
Wall time: 517 ms
CPU times: user 7.9 s, sys: 278 ms, total: 8.18 s
Wall time: 692 ms
CPU times: user 2.22 s, sys: 176 ms, total: 2.4 s
Wall time: 551 ms
CPU times: user 7.68 s, sys: 222 ms, total: 7.9 s
Wall time: 683 ms
CPU times: user 2.37 s, sys: 196 ms, total: 2.57 s
Wall time: 514 ms
CPU times: user 7.64 s, sys: 269 ms, total: 7.91 s
Wall time: 668 ms
CPU times: user 2.41 s, sys: 203 ms, t

In [16]:
%%time
mt2 = peas.scoring_funcs_cython.compute_mean_table_2d_shuffled(hic_corrs, 2, hic_corrs.shape[0], 1)

CPU times: user 2.19 s, sys: 142 ms, total: 2.33 s
Wall time: 499 ms


## Denominators

In [32]:
def naive_denominator(n, start_diagonal, end_diagonal):
    denom_matrix = numpy.zeros((n,n))
    cumulant = 0
    for diag_idx in range(start_diagonal, end_diagonal):
        cumulant += diag_idx - start_diagonal + 1
        for row_idx in range(n-diag_idx):
            col_idx = row_idx + diag_idx
            denom_matrix[row_idx, col_idx] = cumulant
    return denom_matrix

naive_denominator(5, 1, 5)        

array([[ 0.,  1.,  3.,  6., 10.],
       [ 0.,  0.,  1.,  3.,  6.],
       [ 0.,  0.,  0.,  1.,  3.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.]])

In [83]:
reload(peas.scoring)


<module 'peas.scoring' from '/gpfs/data01/glasslab/home/dskola/workspace/PEAS/peas/scoring.py'>

In [79]:
%timeit peas.scoring.compute_denominator_2d(hic_corrs.shape[0], 2)

The slowest run took 8.56 times longer than the fastest. This could mean that an intermediate result is being cached.
1.41 µs ± 1.72 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [82]:
reload(peas.scoring)
peas.scoring.constants.USE_C = True
%timeit peas.scoring.compute_mean_table_2d(hic_corrs, 2)

876 ms ± 8.33 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [81]:
%prun peas.scoring.compute_mean_table_2d(hic_corrs, 2)

In [70]:
# %timeit simple_denominator2(hic_corrs.shape[0])

KeyboardInterrupt: 

In [91]:
%timeit peas.scoring.compute_denominator_2d(5,1)

385 ns ± 4.49 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [93]:
reload(peas.scoring)
peas.scoring.compute_denominator_2d(6,1, 5)

array([[ 0.,  1.,  3.,  6., 10., 15.],
       [ 0.,  0.,  1.,  3.,  6., 10.],
       [ 0.,  0.,  0.,  1.,  3.,  6.],
       [ 0.,  0.,  0.,  0.,  1.,  3.],
       [ 0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.]])

# ToDo: go through all start and end diagonals and straighten out

In [96]:
d = numpy.arange(25).reshape(5,5)
numpy.diag(d, 0)

array([ 0,  6, 12, 18, 24])

In [95]:
d

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [18]:
n = 5
a = numpy.cumsum(numpy.arange(n))
x,y = numpy.meshgrid(a,a)
x,y

(array([[ 0,  1,  3,  6, 10],
        [ 0,  1,  3,  6, 10],
        [ 0,  1,  3,  6, 10],
        [ 0,  1,  3,  6, 10],
        [ 0,  1,  3,  6, 10]]), array([[ 0,  0,  0,  0,  0],
        [ 1,  1,  1,  1,  1],
        [ 3,  3,  3,  3,  3],
        [ 6,  6,  6,  6,  6],
        [10, 10, 10, 10, 10]]))

In [54]:
def simple_denominator2(n, start_diagonal=0):
    return sum([numpy.tri(n,n, -i - start_diagonal)*(i+1) for i in range(n)]).T

In [62]:
%timeit simple_denominator2(5,1)

51.6 µs ± 198 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [67]:
%time simple_denominator2(500,2)

CPU times: user 13.6 s, sys: 719 ms, total: 14.4 s
Wall time: 514 ms


array([[0.00000e+00, 0.00000e+00, 1.00000e+00, ..., 1.23256e+05,
        1.23753e+05, 1.24251e+05],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.22760e+05,
        1.23256e+05, 1.23753e+05],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.22265e+05,
        1.22760e+05, 1.23256e+05],
       ...,
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 1.00000e+00],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00]])

In [66]:
%time peas.scoring.compute_denominator_2d(500,2)

CPU times: user 21.4 ms, sys: 0 ns, total: 21.4 ms
Wall time: 20.7 ms


array([[0.00000e+00, 0.00000e+00, 1.00000e+00, ..., 1.23256e+05,
        1.23753e+05, 1.24251e+05],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.22760e+05,
        1.23256e+05, 1.23753e+05],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.22265e+05,
        1.22760e+05, 1.23256e+05],
       ...,
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 1.00000e+00],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00]])

In [28]:
#simple denominator

n = 5
dm = numpy.zeros((n,n))

x, y = numpy.meshgrid(numpy.arange(5), numpy.arange(5, 0, step=-1))
(x+y) - n

# dm

array([[ 0,  1,  2,  3,  4],
       [-1,  0,  1,  2,  3],
       [-2, -1,  0,  1,  2],
       [-3, -2, -1,  0,  1],
       [-4, -3, -2, -1,  0]])

In [36]:
for pair in zip(*peas.arrayfuncs.my_diag_indices(10, 2)):
    print(pair)

(0, 2)
(1, 3)
(2, 4)
(3, 5)
(4, 6)
(5, 7)
(6, 8)
(7, 9)


In [37]:
for diagonal_idx in range(5):
    print(diagonal_idx)
    for pair in zip(*peas.arrayfuncs.my_diag_indices(5, diagonal_idx)):
        print(pair)
    print()

0
(0, 0)
(1, 1)
(2, 2)
(3, 3)
(4, 4)

1
(0, 1)
(1, 2)
(2, 3)
(3, 4)

2
(0, 2)
(1, 3)
(2, 4)

3
(0, 3)
(1, 4)

4
(0, 4)



In [7]:
n = 5
for k in range(n):
    print(k)
    for row_idx in range(n - k):
        col_idx = row_idx + k
        print(row_idx, col_idx)
        
    print()

0
0 0
1 1
2 2
3 3
4 4

1
0 1
1 2
2 3
3 4

2
0 2
1 3
2 4

3
0 3
1 4

4
0 4



In [8]:
d = numpy.arange(25).reshape((5,5))
d

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [9]:
python_table_sum = peas.scoring.compute_sum_table_2d

In [10]:
n = 5
s = numpy.zeros((n,n))
# 1st diagonal
# for idx in range(n):
#     s[idx, idx] = d[idx, idx]

for k in range(0,n):

    for row_idx in range(n - k):
        this_val = 0
        col_idx = row_idx + k
        print(row_idx, col_idx)
        # current cell
        this_val += d[row_idx, col_idx]
        
        if k >= 1:
            # left cell
            this_val += s[row_idx, col_idx - 1]
            # beneath cell
            this_val += s[row_idx+1, col_idx]
        
            if k >= 2:
                this_val -= s[row_idx +1, col_idx - 1]
        
        s[row_idx, col_idx] = this_val
    print()

0 0
1 1
2 2
3 3
4 4

0 1
1 2
2 3
3 4

0 2
1 3
2 4

0 3
1 4

0 4



In [11]:
def simple_sum_table(d, start_diagonal, end_diagonal):
    n = d.shape[0]
    s = numpy.zeros((n,n))

    for k in range(start_diagonal, end_diagonal+1):
        print(k)
        for row_idx in range(end_diagonal - k + 1):
            this_val = 0
            col_idx = row_idx + k
            print(row_idx, col_idx)
            # current cell
            this_val += d[row_idx, col_idx]

            if k - start_diagonal >= 1:
                # left cell
                this_val += s[row_idx, col_idx - 1]
                # beneath cell
                this_val += s[row_idx+1, col_idx]

                if k - start_diagonal >= 2:
                    this_val -= s[row_idx +1, col_idx - 1]

            s[row_idx, col_idx] = this_val
        print()
    return s

In [12]:
simple_sum_table(d, 0, 4)

0
0 0
1 1
2 2
3 3
4 4

1
0 1
1 2
2 3
3 4

2
0 2
1 3
2 4

3
0 3
1 4

4
0 4



array([[  0.,   7.,  28.,  70., 140.],
       [  0.,   6.,  25.,  64., 130.],
       [  0.,   0.,  12.,  43., 100.],
       [  0.,   0.,   0.,  18.,  61.],
       [  0.,   0.,   0.,   0.,  24.]])

In [13]:
python_table_sum(d)

array([[  0.,   7.,  28.,  70., 140.],
       [  0.,   6.,  25.,  64., 130.],
       [  0.,   0.,  12.,  43., 100.],
       [  0.,   0.,   0.,  18.,  61.],
       [  0.,   0.,   0.,   0.,  24.]])

In [14]:
d

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [15]:
simple_sum_table(d, 1, 4)

1
0 1
1 2
2 3
3 4

2
0 2
1 3
2 4

3
0 3
1 4

4
0 4



array([[ 0.,  1., 10., 34., 80.],
       [ 0.,  0.,  7., 28., 70.],
       [ 0.,  0.,  0., 13., 46.],
       [ 0.,  0.,  0.,  0., 19.],
       [ 0.,  0.,  0.,  0.,  0.]])

In [16]:
python_table_sum(d, 1)

array([[ 0.,  1., 10., 34., 80.],
       [ 0.,  0.,  7., 28., 70.],
       [ 0.,  0.,  0., 13., 46.],
       [ 0.,  0.,  0.,  0., 19.],
       [ 0.,  0.,  0.,  0.,  0.]])

## Test extension

In [17]:
from imp import reload

In [18]:
import peas
reload(peas)

peas.scoring_funcs_cython.compute_sum_table_2d(d.astype(float), 0, 4)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[  0.   7.  28.  70. 140.]
 [  0.   6.  25.  64. 130.]
 [  0.   0.  12.  43. 100.]
 [  0.   0.   0.  18.  61.]
 [  0.   0.   0.   0.  24.]]


array([[  0.,   7.,  28.,  70., 140.],
       [  0.,   6.,  25.,  64., 130.],
       [  0.,   0.,  12.,  43., 100.],
       [  0.,   0.,   0.,  18.,  61.],
       [  0.,   0.,   0.,   0.,  24.]])

In [19]:
import peas
import numpy
d = numpy.arange(25).reshape((5,5))

result = peas.scoring_funcs_cython.compute_sum_table_2d(d.astype(float), 0, 4)
print(result)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[  0.   7.  28.  70. 140.]
 [  0.   6.  25.  64. 130.]
 [  0.   0.  12.  43. 100.]
 [  0.   0.   0.  18.  61.]
 [  0.   0.   0.   0.  24.]]
[[  0.   7.  28.  70. 140.]
 [  0.   6.  25.  64. 130.]
 [  0.   0.  12.  43. 100.]
 [  0.   0.   0.  18.  61.]
 [  0.   0.   0.   0.  24.]]


In [20]:
result = peas.scoring_funcs_cython.compute_sum_table_2d(d.astype(float), 1, 4)


[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[ 0.  1. 10. 34. 80.]
 [ 0.  0.  7. 28. 70.]
 [ 0.  0.  0. 13. 46.]
 [ 0.  0.  0.  0. 19.]
 [ 0.  0.  0.  0.  0.]]


In [23]:
assert 1 < 2 < 3 < 4 < 1 < 8

AssertionError: 